### Imports

In [44]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
from logging import *
from pprint import *
from time import *
import shlex, subprocess

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

%load_ext cython
%load_ext cythonmagic
%matplotlib inline
%matplotlib notebook

from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *
# n.download()

from sklearn_pandas import *

import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.preprocessing import *
from sklearn.cross_validation import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.datasets import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.base import *
from sklearn.datasets.twenty_newsgroups import *
from sklearn.decomposition import *
from sklearn.feature_extraction import *
from sklearn.metrics import *
from sklearn.utils import *

sk.utils.check_random_state(5125)
ran.seed(5125)


The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


### R script adding new features to dataset 

In [15]:
# http://www.mango-solutions.com/wp/2015/10/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/
command = 'Rscript'
path2script = 'insert_features.R'

# Build subprocess command
cmd = [command, path2script]

subprocess.check_output(cmd, universal_newlines=True)

''

In [16]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
data_n.head()

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,POS_sentiment,NEG_sentiment,OverAll_sentiment,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,0,0,4.400,0,0,0,0.700,2,0,0
1,2,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.615,0,0,0,0.577,10,1,0
2,3,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.500,0,2,-2,0.509,14,1,0
3,4,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.667,0,0,0,0.611,4,0,0
4,5,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,...,0,0,2.333,0,0,0,0.857,1,0,0


### Replace 'y' and 'n'   AND easy split

In [17]:
# http://stackoverflow.com/a/17702781
d = {'n': 0, 'y': 1}
data_n = data_n.replace(d)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,POS_sentiment,NEG_sentiment,OverAll_sentiment,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,0,1,...,0,0,4.4,0,0,0,0.7,2,0,0


In [18]:
#norm:http://blog.yhat.com/posts/predicting-customer-churn-with-sklearn.html
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# to_drop = ['State','Area Code','Phone','Churn?']
# churn_feat_space = data_n.drop(to_drop,axis=1)

In [20]:
neu = data_n[["#AUTHID","STATUS","cNEU", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
ext = data_n[["#AUTHID","STATUS","cEXT", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
agr = data_n[["#AUTHID","STATUS","cAGR", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
con = data_n[["#AUTHID","STATUS","cCON", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
opn = data_n[["#AUTHID","STATUS","cOPN", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]

### Split stra. k-fold + selection of columns

In [19]:
# neu = data_n[["#AUTHID","STATUS","cNEU"]]
# train_neu, test_neu, y_train, y_test = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify= neu["cNEU"])
# train_neu, test_neu, y_train, y_test 

In [72]:
train_feat_neu, test_feat_neu, train_class_neu, test_class_neu = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify = neu["cNEU"],random_state= 5152)
print(train_feat_neu.shape, test_feat_neu.shape,train_class_neu.shape ,test_class_neu.shape)

neu_X = train_feat_neu.append(test_feat_neu)
neu_Y = train_class_neu.append(test_class_neu)
train_feat_ext, test_feat_ext, train_class_ext, test_class_ext = sk.cross_validation.train_test_split(ext, ext["cEXT"], train_size = 0.66, stratify = ext["cEXT"],random_state= 5152)
train_feat_agr, test_feat_agr, train_class_agr, test_class_agr = sk.cross_validation.train_test_split(agr, agr["cAGR"], train_size = 0.66, stratify = agr["cAGR"],random_state= 5152)
train_feat_con, test_feat_con, train_class_con, test_class_con = sk.cross_validation.train_test_split(con, con["cCON"], train_size = 0.66, stratify = con["cCON"],random_state= 5152)
train_feat_opn, test_feat_opn, train_class_opn, test_class_opn = sk.cross_validation.train_test_split(opn, opn["cOPN"], train_size = 0.66, stratify = opn["cOPN"],random_state= 5152)

(6545, 11) (3372, 11) (6545,) (3372,)


In [31]:
# mku = sk.pipeline.make_pipeline(SGDClassifier(), LinearSVC(),
#                                # BernoulliNB(), #MultinomialNB()
#                                # KNeighborsClassifier(),
#               RandomForestClassifier(n_jobs=-1),AdaBoostClassifier()) 

### Types of classifiers 
with their configuration

In [75]:
names = ["ttSVC", "ttLinearSVC", "ttMNB", "ttBNB", "ttRF", "ttAD", "ttKNN"]
classifiers = [
    SVC(cache_size=500, kernel = "rbf", decision_function_shape = "ovr"),
    LinearSVC(),
    MultinomialNB(),
    BernoulliNB(),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs=-1),
    AdaBoostClassifier(),
    KNeighborsClassifier()]

### Train + Test many classifiers

In [76]:
for name, clf in zip(names, classifiers):
    clf.fit(train_feat_neu[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]], train_class_neu)
    y_pred = clf.predict(test_feat_neu[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]])
    
    cm = ConfusionMatrix(test_class_neu, y_pred)
    cm.print_stats()
    print("\n NEXT", name, "\n")

Confusion Matrix:

Predicted   0.0  1.0  __all__
Actual                       
0.0         558   37      595
1.0         503   46      549
__all__    1061   83     1144


Overall Statistics:

Accuracy: 0.527972027972
95% CI: (0.49856611934324468, 0.55723376668452085)
No Information Rate: ToDo
P-Value [Acc > NIR]: 1.0
Kappa: 0.0223367316902
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                        0          1
Population                                  1144       1144
P: Condition positive                        595        549
N: Condition negative                        549        595
Test outcome positive                       1061         83
Test outcome negative                         83       1061
TP: True Positive                            558         46
TN: True Negative                             46        558
FP: False Positive                           503         37
FN: False Negative                            37        503
TPR: (Se

In [71]:
eclf = VotingClassifier(estimators=[
        ("svc", ttSVC),
        ("linear_svc", ttLinearSVC)], voting='hard')


for clf, label in zip([ttSVC, ttLinearSVC], ["SVC", "LSVC"]):
    scores = cross_validation.cross_val_score(clf, neu_X[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]], neu_Y, cv=skf, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std()*2, label))
    print(scores)

Accuracy: 0.618 (+/- 0.02) [SVC]
[ 0.62298387  0.60987903  0.61794355  0.60987903  0.61995968  0.62096774
  0.60987903  0.63874874  0.63067608  0.60040363]
Accuracy: 0.576 (+/- 0.20) [LSVC]
[ 0.61995968  0.60887097  0.38810484  0.61189516  0.625       0.63709677
  0.62298387  0.629667    0.64682139  0.37134208]


In [74]:
skf = StratifiedKFold(neu["cNEU"], n_folds=10, shuffle=True, random_state = 5152)
print(skf) 

sklearn.cross_validation.StratifiedKFold(labels=[1 1 1 ..., 1 1 1], n_folds=10, shuffle=True, random_state=5152)


In [35]:
skf = StratifiedKFold(neu["cNEU"], n_folds=10, shuffle=TRUE, random_state = 5152)
print(skf)  

for train_index, test_index in skf:
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    train_feat_neu 
    test_feat_neu
    train_class_neu
    gt = test_class_neu
    
    print(metrics.classification_report(personality_trait_comb, predicted))

In [12]:
# predicted = mku.predict(test_neu[["StringLength", 
#                   "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
#                   "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]])